# 1: Setup and Configuration

****This cell imports necessary libraries and defines the optimized hyperparameters, including the new resolution and unfreeze depth.****

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report
import numpy as np
import os
import matplotlib.pyplot as plt

# --- 0. Configuration and Setup ---
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# --- Data Paths (Must be accurate for Kaggle environment) ---
KAGGLE_ROOT_PATH = '/kaggle/input/animal-faces/afhq'
TRAIN_DIR_FULL = KAGGLE_ROOT_PATH + '/train'
TEST_DIR = KAGGLE_ROOT_PATH + '/val'
CLASS_NAMES = ["cat", "dog", "wild"]
NUM_CLASSES = len(CLASS_NAMES)

# --- Optimized Hyperparameters ---
TARGET_SIZE = (224, 224)    # Increased Resolution (Fix for Lion/Giraffe mix-up)
BATCH_SIZE = 32
VALIDATION_SPLIT_RATIO = 0.105
INPUT_SHAPE = (TARGET_SIZE[0], TARGET_SIZE[1], 3)
MODEL_SAVE_PATH = 'best_efficientnetb0_224_model.keras'

# --- Training and Fine-Tuning Parameters ---
INITIAL_EPOCHS = 10
FINE_TUNE_EPOCHS = 20
INITIAL_LR = 0.001
FINE_TUNE_LR = 1e-5
DROPOUT_RATE = 0.5
DROPOUT_RATE_NEW = 0.4
UNFREEZE_LAYERS = 100 # Deep Fine-Tuning Depth

# 2: Data Preparation and Generators

****sets up the data pipelines, ensuring the images are resized to the optimized $224 \times 224$ resolution and appropriate augmentation is applied.****

In [ ]:
# --- 1. Data Generators Setup ---
train_val_datagen = ImageDataGenerator(
    rescale=1./255, rotation_range=20, horizontal_flip=True, fill_mode='nearest',
    validation_split=VALIDATION_SPLIT_RATIO
)
test_datagen = ImageDataGenerator(rescale=1./255)

# Use the new TARGET_SIZE for all generators
train_generator = train_val_datagen.flow_from_directory(
    TRAIN_DIR_FULL, target_size=TARGET_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', classes=CLASS_NAMES, subset='training'
)
validation_generator = train_val_datagen.flow_from_directory(
    TRAIN_DIR_FULL, target_size=TARGET_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', classes=CLASS_NAMES, subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    TEST_DIR, target_size=TARGET_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', classes=CLASS_NAMES, shuffle=False
)

steps_per_epoch_train = train_generator.samples // BATCH_SIZE
validation_steps_val = validation_generator.samples // BATCH_SIZE

# 3: Model Architecture and the BN Unfreeze Fix

********defines the EfficientNetB0 architecture and applies the critical Batch Normalization (BN) Unfreeze technique to ensure the model learns correctly in the first phase.********

In [ ]:
# --- 2. Build EfficientNetB0 Model (Phase 1: BN Unfrozen) ---
base_model = EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=INPUT_SHAPE
)

# *** CRITICAL FIX: Partial Freezing (Unfreeze BN Layers only) ***
# This allows the BN layers to adapt to the new image statistics (AFHQ) 
# while keeping the core convolutional weights frozen.
base_model.trainable = True 
for layer in base_model.layers:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = False
    # BN layers remain trainable=True (default when trainable=True is set)

# --- Build Custom Head Layers ---
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Two Dense layers with Dropout for robust regularization
x = Dense(512, activation='relu')(x) 
x = Dropout(DROPOUT_RATE)(x) 
x = Dense(256, activation='relu')(x) 
x = Dropout(DROPOUT_RATE_NEW)(x) 

predictions = Dense(NUM_CLASSES, activation='softmax')(x) 
model = Model(inputs=base_model.input, outputs=predictions)

# --- 3. Compile Model for Initial Training ---
model.compile(optimizer=Adam(learning_rate=INITIAL_LR), loss='categorical_crossentropy', metrics=['accuracy'])
print(f"\nModel Ready. Input Resolution: {TARGET_SIZE}. Batch Norm layers are Unfrozen.")

# 4: Initial Training and Deep Fine-Tuning Execution

********Executes the two training phases, including the deep fine-tuning for 100 layers to specialize the model on fine-grained animal features (Lion vs. Dog).********

In [ ]:
# --- 4. Initial Training (Transfer Learning) ---
print("\n--- Starting Initial Training (Phase 1) ---")
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(filepath=MODEL_SAVE_PATH, monitor='val_loss', save_best_only=True, verbose=1)

history_initial = model.fit(
    train_generator, steps_per_epoch=steps_per_epoch_train, epochs=INITIAL_EPOCHS,
    validation_data=validation_generator, validation_steps=validation_steps_val,
    callbacks=[early_stopping, model_checkpoint]
)

# --- 5. Fine-Tuning Setup (Phase 2: Deep Fine-Tuning) ---
print("\n--- Phase 2: Fine-Tuning Setup ---")

# 1. Recreate the Base Model structure to load weights accurately
base_model_ft = EfficientNetB0(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
# Recreate Head Layers (must match Phase 1 structure)
x = base_model_ft.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(DROPOUT_RATE)(x) 
x = Dense(256, activation='relu')(x)
x = Dropout(DROPOUT_RATE_NEW)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model_ft.input, outputs=predictions)

# 2. Load best weights from Phase 1 checkpoint
try:
    model.load_weights(MODEL_SAVE_PATH) 
    print(f"Successfully loaded best weights from: {MODEL_SAVE_PATH}")
except Exception as e:
    print(f"Error loading weights: {e}")

# 3. Unfreeze 100 layers for specialized learning
base_model_ft.trainable = True
num_base_layers = len(base_model_ft.layers)
freeze_until = num_base_layers - UNFREEZE_LAYERS

# Freeze the first part, unfreeze the final N layers
for layer in base_model_ft.layers[:freeze_until]:
    layer.trainable = False
for layer in base_model_ft.layers[freeze_until:]:
    layer.trainable = True

print(f"Starting Fine-Tuning with {UNFREEZE_LAYERS} layers unfrozen.")

# 4. Recompile with low learning rate
model.compile(optimizer=Adam(learning_rate=FINE_TUNE_LR), loss='categorical_crossentropy', metrics=['accuracy'])

# --- 7. Fine-Tuning Training --- 
print("\n--- Starting Fine-Tuning Training (Phase 2) ---")
history_fine = model.fit(
    train_generator, steps_per_epoch=steps_per_epoch_train, epochs=FINE_TUNE_EPOCHS,
    validation_data=validation_generator, validation_steps=validation_steps_val,
    callbacks=[early_stopping, model_checkpoint]
)

# 5: Final Evaluation and Deployment Preparation

******performs the final evaluation on the test set, prints the classification report, and saves the final model for API deployment******

In [ ]:
# --- 8. Final Evaluation, Report, and Visualization ---

# 1. Final Evaluation on Test Set
print("\n--- Final Model Evaluation on Test Set ---")
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // BATCH_SIZE)
print(f"Test Accuracy: {test_acc:.4f}")

# 2. Detailed Classification Report (Crucial for verifying Lion/Dog fix)
print("\n--- Detailed Classification Report ---")
test_generator.reset()
Y_pred = model.predict(test_generator, steps=test_generator.samples // BATCH_SIZE + 1)
y_pred_classes = np.argmax(Y_pred, axis=1)
y_true_classes = test_generator.classes[:len(y_pred_classes)]

report = classification_report(y_true_classes, y_pred_classes, target_names=CLASS_NAMES, zero_division=0)
print(report)

# 3. Final Model Save for Deployment
SAVED_MODEL_DIR = './final_efficientnetb0_224_saved_model'
tf.saved_model.save(model, SAVED_MODEL_DIR)
print(f"\n✨ Final Model Saved for API Deployment: {SAVED_MODEL_DIR}")

# Optional: Plotting code goes here after the training is complete